In [39]:
# 两节点的连边
class Edge(object):
    def __init__(self, u, v, w):
        self.source = u
        self.sink = v  
        self.capacity = w
    def __repr__(self):
        return "%s->%s:%s" % (self.source, self.sink, self.capacity)

# 网络流
class NetworkFlow(object):
    def __init__(self):
        self.adj = {}  # 边集合
        self.flow = {} # 边的流量
 
    # 添加顶点
    def add_vertex(self, vertex):
        self.adj[vertex] = []
    
    # 获取从 v 出发的边集合
    def get_edges(self, v):
        return self.adj[v]

    # 添加边
    def add_edge(self, u, v, w=0):
        if u == v:
            raise ValueError("u == v")
        edge = Edge(u,v,w)
        redge = Edge(v,u,0)
        edge.redge = redge
        redge.redge = edge
        self.adj[u].append(edge)
        self.adj[v].append(redge)
        self.flow[edge] = 0
        self.flow[redge] = 0
 
    # 寻找一条增广路径
    def find_path(self, source, sink, path):
        if source == sink:
            return path
        for edge in self.get_edges(source):
            residual = edge.capacity - self.flow[edge]            
            if residual > 0 and edge not in path:
                result = self.find_path( edge.sink, sink, path + [edge]) 
                if result != None:
                    return result        

    # 计算最大流
    def max_flow(self, source, sink):
        path = self.find_path(source, sink, [])
        while path != None:            
            residuals = [edge.capacity - self.flow[edge] for edge in path]
            flow = min(residuals)
            for edge in path:
                self.flow[edge] += flow
                self.flow[edge.redge] -= flow
            path = self.find_path(source, sink, [])
        return sum(self.flow[edge] for edge in self.get_edges(source))

In [40]:
def load_data(path):    
    with open(path, "rt") as f:
        i = 1
        while True:            
            if i > 3:
                line = f.readline()
                if not line:
                    break                                
#                 print(line, end='')
                split_line = line.split()
                N = int(split_line[0])
                M = int(split_line[1])
                print("N = %d, M = %d" %(N, M))
                g = NetworkFlow()
                g.add_vertex("s")  # 添加源点 s
                J_vertex = []
                C_vertex = []
                for j in range(N):
                    g.add_vertex("j" + str(j + 1))  # 添加 job 顶点集合 J
                    J_vertex.append("j" + str(j + 1))                    
                for j in range(M):
                    g.add_vertex("c" + str(j + 1))  # 添加 computer 顶点集合 C
                    C_vertex.append("c" + str(j + 1))                    
                g.add_vertex("t")  # 添加汇点 t
                linked_c_vertex = []
                for j in range(N):
                    line = f.readline()
#                     print(line, end='')
                    g.add_edge("s", "j" + str(j + 1), 1)  # 添加 s 和 J 中各顶点的连边，容量为1
                    split_line = line.split()
                    linked_c_vertex.append(split_line)
                    for k in range(len(split_line)):
                        g.add_edge("j" + str(j + 1), "c" + split_line[k], 1)  # 添加 J 和 C 之间的连边，容量为1                
                for j in range(M):
                    g.add_edge("c" + str(j + 1), "t", N)  # 添加 C 中各顶点与 t 的连边，初始容量为 N
                max_flow = g.max_flow('s', 't')                            
                if max_flow != N:
                    print("Thers's no solution !")
                else:
                    left = 1
                    right = N
                    print("ct capacity: ", N)
                    print("max flow: ", max_flow)
                    min_max_load = 0
                    for c in C_vertex:
                        for edge in g.get_edges(c):
                            if min_max_load < g.flow[edge]:
                                min_max_load = g.flow[edge]
                    print("min max load: ", min_max_load)
                    while left < right:             # 二分查找最小最大负载       
                        mid = int(left + (right - left) / 2)
                        print("ct capacity: ", mid)  # C 中各顶点与 t 连边的修改后的容量
                        g = NetworkFlow()
                        g.add_vertex("s")  # 添加源点 s
                        for j in J_vertex:
                            g.add_vertex(j) # 添加 job 顶点集合 J
                        for c in C_vertex:
                            g.add_vertex(c) # 添加 computer 顶点集合 C
                        g.add_vertex("t")  # 添加汇点 t
                        for j in range(len(linked_c_vertex)):
                            g.add_edge("s", "j" + str(j + 1), 1)  # 添加 s 和 J 中各顶点的连边，容量为1
                            c_list = linked_c_vertex[j]
                            for k in range(len(c_list)):
                                g.add_edge("j" + str(j + 1), "c" + c_list[k], 1)  # 添加 J 和 C 之间的连边，容量为1
                        for j in range(M):
                            g.add_edge("c" + str(j + 1), "t", mid)  # 添加 C 中各顶点与 t 的连边，初始容量为 N
                        if mid * M < N:    # 如果C 中各顶点与 t 连边的容量之和小于 job 的总数，无解
                            left = mid + 1
                            continue
                        max_flow = g.max_flow('s', 't')     
                        print("max flow: ", max_flow)
                        if max_flow != N:
                            left = mid + 1
                        else:
                            right = mid    
                            min_max_load = 0
                            for c in C_vertex:
                                for edge in g.get_edges(c):
    #                                 print(edge)                                
    #                                 print("flow: ", g.flow[edge])
                                    if min_max_load < g.flow[edge]:
                                        min_max_load = g.flow[edge]
                            print("min max load: ", min_max_load)
                    print("The minimize max load is: ", min_max_load)
                    print()
                f.readline()                
            else:
                f.readline()
            i += 1

In [41]:
load_data("./problem8.data")

N = 4, M = 2
ct capacity:  4
max flow:  4
min max load:  2
ct capacity:  2
max flow:  4
min max load:  2
ct capacity:  1
The minimize max load is:  2

N = 7, M = 49
ct capacity:  7
max flow:  7
min max load:  1
ct capacity:  4
max flow:  7
min max load:  1
ct capacity:  2
max flow:  7
min max load:  1
ct capacity:  1
max flow:  7
min max load:  1
The minimize max load is:  1

N = 27, M = 29
ct capacity:  27
max flow:  27
min max load:  2
ct capacity:  14
max flow:  27
min max load:  2
ct capacity:  7
max flow:  27
min max load:  2
ct capacity:  4
max flow:  27
min max load:  2
ct capacity:  2
max flow:  27
min max load:  2
ct capacity:  1
max flow:  25
The minimize max load is:  2

N = 35, M = 13
ct capacity:  35
max flow:  35
min max load:  5
ct capacity:  18
max flow:  35
min max load:  5
ct capacity:  9
max flow:  35
min max load:  5
ct capacity:  5
max flow:  35
min max load:  5
ct capacity:  3
max flow:  35
min max load:  3
ct capacity:  2
The minimize max load is:  3

N = 16, M =